# Knowledge Store 使用指南

# 1. 初始化


In [2]:
from tests.conftest import MockEmbedding
from tablestore_for_agent_memory.knowledge.knowledge_store import KnowledgeStore
import tablestore
import os

endpoint = os.getenv("tablestore_end_point")
instance_name = os.getenv("tablestore_instance_name")
access_key_id = os.getenv("tablestore_access_key_id")
access_key_secret = os.getenv("tablestore_access_key_secret")

# 创建 tablestore 的 sdk client
tablestore_client = tablestore.OTSClient(
    endpoint,
    access_key_id,
    access_key_secret,
    instance_name,
    retry_policy=tablestore.WriteRetryPolicy(),
)

# 需要索引哪些meta字段，下面进行举例。document_id/tenant_id/text/vector_content这些必备字段不需要设置。
search_index_schema = [
    tablestore.FieldSchema("user_id", tablestore.FieldType.KEYWORD),
    tablestore.FieldSchema("meta_string", tablestore.FieldType.KEYWORD),
    tablestore.FieldSchema("meta_long", tablestore.FieldType.LONG),
    tablestore.FieldSchema("meta_double", tablestore.FieldType.DOUBLE),
    tablestore.FieldSchema("meta_boolean", tablestore.FieldType.BOOLEAN),
]

# 假设我们使用的embedding模型是维度=4
embedding_model = MockEmbedding(4)

knowledge_store = KnowledgeStore(
    tablestore_client=tablestore_client,
    vector_dimension=4,  # 需要与embedding模型的维度相等，上述我们使用的embedding模型维度为4。大多数生产使用的embedding模型维度为512/768等
    enable_multi_tenant=True,  # 是否开启多租户能力，这里以开启多租户为例。
    search_index_schema=search_index_schema,
)

## 1.1 多租户能力说明

在多租户场景中，租户可以是知识库、用户、组织等，具体可以参考业务场景。通常来说使用用户id或者知识库id当做租户id有通用性。


##### 1.1.1 多租户场景举例：

以用户id为例，该数据库中存储了不同用户的数据，在每次查询时候，一定会带上用户id的filter限制，来查询这个用户的数据里最相似的文档，则该场景是适合多租户能力的。开启多租户后，会在多元索引里设置路由，同时开启一些向量检索的多租户优化能力，降低查询延时，提高召回率。

##### 1.1.2 非多租户场景举例：

数据库中只存储了一个用户的某一个领域的全部数据，查询时候，不会添加用户id、知识库id等租户特征的filter条件，每次都是在全部数据中查找最相似的文档，该场景则适合非多租户。


## 1.2 初始化表

In [3]:
# 创建表(包括多元索引，后续索引结构可以在控制台上修改)
# 仅需执行一次
knowledge_store.init_table()

tablestore table:[knowledge] already exists
tablestore search index[knowledge_search_index_name] already exists


In [ ]:
# 删除表（内部接口，仅供测试使用）
knowledge_store._delete_table()

 # 2. 文档管理

本章节主要介绍文档的增删改查。

## 2.1 声明 Document

In [4]:
from tablestore_for_agent_memory.base.base_knowledge_store import Document

# 声明一个Document
# document_id和tenant_id(租户id)唯一确认一行数据。 如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。
document = Document(document_id="1", tenant_id="user_id_1")
# 文档的文本内容
document.text = "123"
# 文档的向量内容，格式为 list[float], 此处我们调用embedding模型将文档的文本内容转成向量
document.embedding = embedding_model.embedding(document.text)
# 声明一些额外的meta信息
document.metadata["meta_string"] = "123"
document.metadata["meta_bool"] = True
document.metadata["meta_double"] = 123.456
document.metadata["meta_long"] = 123456
document.metadata["meta_bytes"] = bytearray("China", encoding="utf8")

## 2.2 创建、更新、查询、删除 Document


In [6]:
# 创建(不管是否存在，都会进行覆盖写入)
knowledge_store.put_document(document)
# 查询 （如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。）
document = knowledge_store.get_document(document_id="1", tenant_id="user_id_1")
print(document)
print(document.metadata["meta_string"])

Document(document_id='1', tenant_id='user_id_1', text='123', embedding=[0.7501716876718638, 0.7501716876718638, 0.7501716876718638, 0.7501716876718638], metadata={'meta_bool': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 123.456, 'meta_long': 123456, 'meta_string': '123'})
123


In [7]:
document_for_update = Document(document_id="1", tenant_id="user_id_1")
document_for_update.metadata["meta_string"] = "updated from 123"
# 增量更新 (无更新场景，推荐使用 put_document 进行覆盖写入) 此处我们将上述文档的"meta_string"字段修改为一个新值
knowledge_store.update_document(document_for_update)
# 再次查询该文档，得到更新后的值
document = knowledge_store.get_document(document_id="1", tenant_id="user_id_1")
print(document.metadata["meta_string"])

updated from 123


In [8]:
# 批量获取文档
id_list = ["0", "1", "2"]
# 如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。
documents_by_batch_get = knowledge_store.get_documents(document_id_list=id_list, tenant_id="user_id_1")
print("第0个文档:", documents_by_batch_get[0])
print("第1个文档:", documents_by_batch_get[1])
print("第2个文档:", documents_by_batch_get[2])

第0个文档: None
第1个文档: Document(document_id='1', tenant_id='user_id_1', text='123', embedding=[0.7501716876718638, 0.7501716876718638, 0.7501716876718638, 0.7501716876718638], metadata={'meta_bool': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 123.456, 'meta_long': 123456, 'meta_string': 'updated from 123'})
第2个文档: None


In [9]:
# 删除（如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。）
knowledge_store.delete_document(document_id="1", tenant_id="user_id_1")
print(knowledge_store.get_document(document_id="1", tenant_id="user_id_1"))

None


In [10]:
# 根据租户删除数据
knowledge_store.delete_document_by_tenant(tenant_id="user_id_1")
print(knowledge_store.get_document(document_id="1", tenant_id="user_id_1"))

None


## 2.3 搜索文档

### 2.3.1 写入样例数据
 

In [5]:
from faker import Faker
import random

fk_data = Faker(locale="zh_CN")


def random_document() -> Document:
    tenant_id = random.choice(["1", "2"])
    document = Document(document_id=fk_data.uuid4(), tenant_id=tenant_id)
    # 文档的文本内容
    document.text = " ".join(
        random.choices(
            ["abc", "def", "ghi", "abcd", "adef", "abcgh", "apple", "banana", "cherry"], k=random.randint(1, 10)
        )
    )
    # 文档的向量内容，格式为 list[float], 此处我们调用embedding模型将文档的文本内容转成向量
    document.embedding = embedding_model.embedding(document.text)
    # 声明一些额外的meta信息
    document.metadata["meta_string"] = fk_data.name_female()
    document.metadata["meta_long"] = random.randint(0, 88888888)
    document.metadata["meta_double"] = random.uniform(1.0, 2.0)
    document.metadata["meta_boolean"] = random.choice([False, True])
    document.metadata["meta_bytes"] = bytearray(fk_data.city_name(), encoding="utf8")
    return document


total_count = 100
for i in range(total_count):
    document = random_document()
    knowledge_store.put_document(document)

### 2.3.2 向量检索


In [13]:
from tablestore_for_agent_memory.base.filter import Filters

# 查询和“abc”相似的文档
query_vector = embedding_model.embedding("abc")

response = knowledge_store.vector_search(
    query_vector=query_vector,
    top_k=20,
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
)
print(len(response.hits))

# 展示前2个文档
for hit in response.hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

20
文档内容: Document(document_id='3dfdcff0-b008-4bfd-94bf-ac15f8131daa', tenant_id='1', text='banana banana ghi cherry banana ghi abc abc ghi cherry', embedding=None, metadata={'meta_boolean': True, 'meta_double': 1.9409917690183969, 'meta_long': 76911423, 'meta_string': '顾秀云'})
文档分数: 0.9953961968421936
文档内容: Document(document_id='1778574e-f7e5-4bd5-ab84-1a6cdc097bc9', tenant_id='1', text='abc apple adef ghi def adef abcd abcd', embedding=None, metadata={'meta_boolean': True, 'meta_double': 1.965595902602045, 'meta_long': 13476874, 'meta_string': '易建华'})
文档分数: 0.9929059147834778


### 2.3.3 全文检索

In [7]:
# 全文检索“abc”相关的文档

# next_token 可以进行连续翻页
response = knowledge_store.full_text_search(
    query="abc",
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    limit=20,
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
)

print(len(response.hits))
print("next_token", response.next_token)

# 展示前2个文档
for hit in response.hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

20
next_token CAESBgoEGgIIARgBImAKCQFf1bTmIrkKQApTA04AAAAxUzY1NjY2NTM5MzczODMxMzgyZDYzNjU2MTYzMmQzNDY2MzU2NTJkMzgzOTYzMzEyZDM0NjIzMDM4NjI2NTY2NjM2MTM4NjE2NS5TMzE=
文档内容: Document(document_id='79e2b695-e684-4bf3-b149-797710b480be', tenant_id='1', text='abc abc adef', embedding=None, metadata={'meta_boolean': True, 'meta_double': 1.3120816914236506, 'meta_long': 37053873, 'meta_string': '尹敏'})
文档分数: 3.9219722747802734
文档内容: Document(document_id='fcad2bd8-7365-453c-a88c-05f5056c1a74', tenant_id='1', text='cherry abcd abc abc abcd', embedding=None, metadata={'meta_boolean': True, 'meta_double': 1.8225253209919727, 'meta_long': 46749064, 'meta_string': '张勇'})
文档分数: 3.7818353176116943


### 2.3.4 通用检索



In [14]:
response = knowledge_store.search_documents(
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    limit=20,
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
    meta_data_to_get=["text", "meta_string", "meta_boolean"],  # 只返回这三个字段,不填则默认返回索引里有的字段
)

print(len(response.hits))
print("next_token", response.next_token)

# 展示前2个文档
for hit in response.hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

20
next_token CAESBgoEIgIIABgAIlUKUwNOAAAAMVMzNzM5NjUzMjYyMzYzOTM1MmQ2NTM2MzgzNDJkMzQ2MjY2MzMyZDYyMzEzNDM5MmQzNzM5MzczNzMxMzA2MjM0MzgzMDYyNjUuUzMx
文档内容: Document(document_id='00ca52fa-bbd8-4bb5-b047-fe81d43ae5a7', tenant_id='1', text='def abcgh', embedding=None, metadata={'meta_boolean': True, 'meta_string': '张浩'})
文档分数: None
文档内容: Document(document_id='0d5536ec-aafa-4f58-8809-6b3ba2cfa355', tenant_id='1', text='banana def cherry abcgh abcgh abcd ghi abc', embedding=None, metadata={'meta_boolean': True, 'meta_string': '高建华'})
文档分数: None


In [16]:
# 翻页获取所有数据, 每页获取limit=3.
total_document_hits = []
next_token = None
while True:
    response = knowledge_store.search_documents(
        tenant_id="1",
        limit=3,
        meta_data_to_get=[
            "text",
            "meta_string",
            "meta_boolean",
        ],  # 只返回这三个字段,不填则默认返回索引里有的字段
        next_token=next_token,
    )
    total_document_hits.extend(response.hits)
    if next_token is None:
        break
print("total docs", len(total_document_hits))

total docs 3
